In [1]:
# Packages used in the script
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import io
import gzip
import spacy
import re
from contractions import contractions_dict

In [ ]:
df_elec = pd.read_csv (r'Path where the CSV file is stored\df_electronics.csv')

In [669]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import regularizers
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers


In [670]:
max_features = 1800
maxlen = 50

In [671]:
word_tokenizer = Tokenizer()

X = df_elec['sentences']
word_tokenizer.fit_on_texts(X)

X_encoded = word_tokenizer.texts_to_sequences(X) 

X_padded = pad_sequences(X_encoded, maxlen=maxlen, padding='post', truncating='post')

X = X_padded.astype('int')
X = np.array(X)

In [674]:
# We have in total 5 tags
import itertools
tags = set(itertools.chain.from_iterable(df_elec.IOB))
print(tags)
n_tags = len(tags)

{'B_pos', 'B_neg', 'I_pos', 'O', 'I_neg'}


In [675]:
tag2idx = {t: i for i, t in enumerate(tags)}
tag2idx['O']

3

In [676]:
y = [[tag2idx[w] for w in s] for s in df_elec['IOB']]


y = pad_sequences(y, maxlen=maxlen, padding='post', truncating='post', value = tag2idx['O'])


In [677]:
y = to_categorical(y, dtype = 'int', num_classes= 5)
                   


In [679]:
embedding_size  = 300  
vocabulary_size = len(word_tokenizer.word_index) + 1

In [680]:
embedding_weights = np.zeros((vocabulary_size, embedding_size))

In [681]:
# Total number of words
word2id = word_tokenizer.word_index

In [682]:
X[5]

array([  17,  158,    7, 1752,    4,   87,  184,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0])

In [684]:
print(X_padded.shape)
print(y.shape)

(5000, 50)
(5000, 50, 5)


In [685]:
# Split in train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [686]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(4000, 50) (4000, 50, 5)
(1000, 50) (1000, 50, 5)


In [687]:
len(y_train), len(y_test)

(4000, 1000)

In [689]:
opt = optimizers.Adam(learning_rate=0.01, decay=1e-6)

In [690]:
tf.random.set_seed(1234)

In [691]:
lstm_model = keras.Sequential()
lstm_model.add(tf.keras.layers.Embedding(vocabulary_size, embedding_size, input_length = maxlen)) #The embedding layer
lstm_model.add(tf.keras.layers.LSTM(64, dropout=0.6, return_sequences=True)) #Our LSTM layer
lstm_model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(5, activation='softmax')))
lstm_model.summary()

lstm_model.compile(opt, "categorical_crossentropy", metrics=["accuracy"])
history = lstm_model.fit(X_train, y_train, batch_size = 128, epochs=10) 

Model: "sequential_56"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_56 (Embedding)     (None, 50, 300)           2563500   
_________________________________________________________________
lstm_56 (LSTM)               (None, 50, 64)            93440     
_________________________________________________________________
time_distributed_45 (TimeDis (None, 50, 5)             325       
Total params: 2,657,265
Trainable params: 2,657,265
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
32/32 [==============================] - 4s 82ms/step - loss: 0.2311 - accuracy: 0.9454
Epoch 2/10
32/32 [==============================] - 3s 81ms/step - loss: 0.1007 - accuracy: 0.9698
Epoch 3/10
32/32 [==============================] - 3s 80ms/step - loss: 0.0894 - accuracy: 0.9717
Epoch 4/10
32/32 [==============================] - 3s 81ms/step - loss: 0.0

In [692]:
loss, accuracy = lstm_model.evaluate(X_test, y_test, verbose = 1)
print('Loss: {0},\nAccuracy: {1}'.format(loss, accuracy))

32/32 [==============================] - 1s 8ms/step - loss: 0.1232 - accuracy: 0.9667
Loss: 0.12321771681308746,
Accuracy: 0.9667199850082397


In [693]:
# Predict
y_pred_lstm = lstm_model.predict(X_test)

# Make hard classes 
y_pred_lstm[y_pred_lstm > 0.5] = 1
y_pred_lstm[y_pred_lstm <= 0.5] = 0

y_pred_lstm = np.array(y_pred_lstm.astype('int'))

pred_lstm = np.array(tf.argmax(y_pred_lstm, axis = -1))
y_true = np.array(tf.argmax(y_test, axis = -1))

In [694]:
# Confusion matrix LSTM model - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.multilabel_confusion_matrix.html
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(y_test.flatten(), y_pred_lstm.flatten())

array([[[ 48136,   1864],
        [  1464, 198536]],

       [[198536,   1464],
        [  1864,  48136]]])

In [695]:
# Classification report LSTM model
from sklearn.metrics import classification_report
print(classification_report(y_true.flatten(), pred_lstm.flatten()))

              precision    recall  f1-score   support

           0       0.40      0.44      0.42      1150
           1       0.41      0.31      0.36       194
           2       0.21      0.06      0.10       175
           3       0.98      0.98      0.98     48449
           4       1.00      0.03      0.06        32

    accuracy                           0.96     50000
   macro avg       0.60      0.37      0.38     50000
weighted avg       0.96      0.96      0.96     50000



In [696]:
# Precision LSTM model - https://www.tensorflow.org/api_docs/python/tf/keras/metrics/Precision
m = tf.keras.metrics.Precision()
m.update_state(y_test, y_pred_lstm)
precision = m.result().numpy()
print(precision)

0.9704839


In [697]:
# Recall LSTM model - https://www.tensorflow.org/api_docs/python/tf/keras/metrics/Recall
m = tf.keras.metrics.Recall()
m.update_state(y_test, y_pred_lstm)
recall = m.result().numpy()
print(recall)

0.96272


In [698]:
# F1-score LSTM model - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html
F1 = 2 * (precision * recall) / (precision + recall)
print(F1)

0.9665863183384731


In [699]:
#Bidirectional LSTM https://github.com/sergiovirahonda/TweetsSentimentAnalysis/blob/main/TweetsSentimentPredictions.ipynb
bilstm_model = keras.Sequential()
bilstm_model.add(tf.keras.layers.Embedding(vocabulary_size, embedding_size, input_length = maxlen))
bilstm_model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,dropout=0.6, return_sequences=True)))
bilstm_model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(5, activation='softmax')))
bilstm_model.summary()



bilstm_model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=["accuracy"])

history = bilstm_model.fit(X_train, y_train, batch_size = 128, epochs=10)

Model: "sequential_57"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_57 (Embedding)     (None, 50, 300)           2563500   
_________________________________________________________________
bidirectional_21 (Bidirectio (None, 50, 128)           186880    
_________________________________________________________________
time_distributed_46 (TimeDis (None, 50, 5)             645       
Total params: 2,751,025
Trainable params: 2,751,025
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
32/32 [==============================] - 6s 124ms/step - loss: 0.2052 - accuracy: 0.9398
Epoch 2/10
32/32 [==============================] - 4s 122ms/step - loss: 0.0995 - accuracy: 0.9702
Epoch 3/10
32/32 [==============================] - 4s 120ms/step - loss: 0.0887 - accuracy: 0.9719
Epoch 4/10
32/32 [==============================] - 4s 125ms/step - loss:

In [700]:
loss, accuracy = bilstm_model.evaluate(X_test, y_test, verbose = 1)
print("Loss: {0},\nAccuracy: {1}".format(loss, accuracy))

32/32 [==============================] - 1s 14ms/step - loss: 0.1201 - accuracy: 0.9656
Loss: 0.12014035880565643,
Accuracy: 0.9656400084495544


In [701]:
# Predict
y_pred_bilstm = bilstm_model.predict(X_test)

# Make hard classes
y_pred_bilstm[y_pred_bilstm > 0.5] = 1
y_pred_bilstm[y_pred_bilstm <= 0.5] = 0

y_pred_bilstm = np.array(y_pred_bilstm.astype('int'))

pred_bilstm = np.array(tf.argmax(y_pred_bilstm, axis = -1))
y_true = np.array(tf.argmax(y_test, axis = -1))

In [702]:
# Confusion matrix LSTM model - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.multilabel_confusion_matrix.html
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(y_test.flatten(), y_pred_bilstm.flatten())

array([[[ 48177,   1823],
        [  1593, 198407]],

       [[198407,   1593],
        [  1823,  48177]]])

In [703]:
# Classification report LSTM model
from sklearn.metrics import classification_report
print(classification_report(y_true.flatten(), pred_bilstm.flatten()))

              precision    recall  f1-score   support

           0       0.40      0.45      0.42      1150
           1       0.48      0.26      0.33       194
           2       0.18      0.07      0.11       175
           3       0.98      0.98      0.98     48449
           4       0.50      0.03      0.06        32

    accuracy                           0.96     50000
   macro avg       0.51      0.36      0.38     50000
weighted avg       0.96      0.96      0.96     50000



In [704]:
# Precision LSTM model - https://www.tensorflow.org/api_docs/python/tf/keras/metrics/Precision
m = tf.keras.metrics.Precision()
m.update_state(y_test, y_pred_bilstm)
precision = m.result().numpy()
print(precision)

0.9679928


In [705]:
# Recall LSTM model - https://www.tensorflow.org/api_docs/python/tf/keras/metrics/Recall
m = tf.keras.metrics.Recall()
m.update_state(y_test, y_pred_bilstm)
recall = m.result().numpy()
print(recall)

0.96354


In [706]:
# F1-score LSTM model - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html
F1 = 2 * (precision * recall) / (precision + recall)
print(F1)

0.9657612551651337


In [707]:
#Add an attention-layer
from tensorflow.keras.layers import Dense, Lambda, Dot, Activation, Concatenate
from tensorflow.keras.layers import Layer


class Attention(Layer):

    def __init__(self, units=128, **kwargs):
        self.units = units
        super().__init__(**kwargs)

    def __call__(self, inputs):
        """
        Many-to-one attention mechanism for Keras.
        @param inputs: 3D tensor with shape (batch_size, time_steps, input_dim).
        @return: 2D tensor with shape (batch_size, 128)
        @author: felixhao28, philipperemy.
        """
        hidden_states = inputs
        hidden_size = int(hidden_states.shape[2])
        # Inside dense layer
        #              hidden_states            dot               W            =>           score_first_part
        # (batch_size, time_steps, hidden_size) dot (hidden_size, hidden_size) => (batch_size, time_steps, hidden_size)
        # W is the trainable weight matrix of attention Luong's multiplicative style score
        score_first_part = Dense(hidden_size, use_bias=False, name='attention_score_vec')(hidden_states)
        #            score_first_part           dot        last_hidden_state     => attention_weights
        # (batch_size, time_steps, hidden_size) dot   (batch_size, hidden_size)  => (batch_size, time_steps)
        h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,), name='last_hidden_state')(hidden_states)
        score = Dot(axes=[1, 2], name='attention_score')([h_t, score_first_part])
        attention_weights = Activation('softmax', name='attention_weight')(score)
        # (batch_size, time_steps, hidden_size) dot (batch_size, time_steps) => (batch_size, hidden_size)
        context_vector = Dot(axes=[1, 1], name='context_vector')([hidden_states, attention_weights])
        pre_activation = Concatenate(name='attention_output')([context_vector, h_t])
        attention_vector = Dense(self.units, use_bias=False, activation='tanh', name='attention_vector')(pre_activation)
        return attention_vector

    def get_config(self):
        return {'units': self.units}

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [708]:
# Attention-based LSTM model
import os
os.environ['TF_KERAS'] = '1'
from keras_self_attention import SeqSelfAttention

attentionlstm_model = keras.Sequential()
attentionlstm_model.add(tf.keras.layers.Embedding(vocabulary_size, embedding_size, input_length = maxlen))# weights = [embedding_weights], trainable = True))
attentionlstm_model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)))

attentionlstm_model.add(Layer(SeqSelfAttention(attention_activation='sigmoid')))

attentionlstm_model.add(tf.keras.layers.Dense(5, 'softmax'))

attentionlstm_model.summary()
attentionlstm_model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=["accuracy"])

history = attentionlstm_model.fit(X_train, y_train, batch_size = 128, epochs=10)

Model: "sequential_58"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_58 (Embedding)     (None, 50, 300)           2563500   
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 50, 128)           186880    
_________________________________________________________________
layer_11 (Layer)             (None, 50, 128)           0         
_________________________________________________________________
dense_58 (Dense)             (None, 50, 5)             645       
Total params: 2,751,025
Trainable params: 2,751,025
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
32/32 [==============================] - 6s 111ms/step - loss: 0.2061 - accuracy: 0.9430
Epoch 2/10
32/32 [==============================] - 4s 111ms/step - loss: 0.0984 - accuracy: 0.9701
Epoch 3/10
32/32 [=======================

In [709]:
loss, accuracy = attentionlstm_model.evaluate(X_test, y_test, verbose =1)
print("Loss: {0},\nAccuracy: {1}".format(loss, accuracy))

32/32 [==============================] - 1s 15ms/step - loss: 0.1308 - accuracy: 0.9675
Loss: 0.1308419555425644,
Accuracy: 0.9675400257110596


In [710]:
# Predict
y_pred_attentionlstm = attentionlstm_model.predict(X_test)

# Make hard classes
y_pred_attentionlstm[y_pred_attentionlstm > 0.5] = 1
y_pred_attentionlstm[y_pred_attentionlstm <= 0.5] = 0

y_pred_attentionlstm = np.array(y_pred_attentionlstm.astype('int'))

pred_attentionlstm = np.array(tf.argmax(y_pred_attentionlstm, axis = -1))
y_true = np.array(tf.argmax(y_test, axis = -1))

In [711]:
# Confusion matrix LSTM model - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.multilabel_confusion_matrix.html
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(y_test.flatten(), y_pred_attentionlstm.flatten())

array([[[ 48303,   1697],
        [  1530, 198470]],

       [[198470,   1530],
        [  1697,  48303]]])

In [712]:
# Classification report LSTM model
from sklearn.metrics import classification_report
print(classification_report(y_true.flatten(), pred_attentionlstm.flatten()))

              precision    recall  f1-score   support

           0       0.42      0.36      0.39      1150
           1       0.40      0.15      0.22       194
           2       0.28      0.06      0.10       175
           3       0.98      0.99      0.98     48449
           4       0.25      0.03      0.06        32

    accuracy                           0.97     50000
   macro avg       0.46      0.32      0.35     50000
weighted avg       0.96      0.97      0.96     50000



In [713]:
# Precision LSTM model - https://www.tensorflow.org/api_docs/python/tf/keras/metrics/Precision
m = tf.keras.metrics.Precision()
m.update_state(y_test, y_pred_attentionlstm)
precision = m.result().numpy()
print(precision)

0.96929747


In [714]:
# Recall LSTM model - https://www.tensorflow.org/api_docs/python/tf/keras/metrics/Recall
m = tf.keras.metrics.Recall()
m.update_state(y_test, y_pred_attentionlstm)
recall = m.result().numpy()
print(recall)

0.96606


In [715]:
# F1-score LSTM model - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html
F1 = 2 * (precision * recall) / (precision + recall)
print(F1)

0.9676760422187805
